In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from google.colab import drive

os.chdir("drive/My Drive/IIITH/Advanced_NLP//Assignment_2")

In [2]:
#Eng_corpus = open("english_sampled.txt","r", encoding='utf-8')
#Eng_corpus = Eng_corpus.read().splitlines()
#print('English Corpus length is ',len(Eng_corpus))

#Hindi_corpus = open("hindi_sampled.txt","r", encoding='utf-8')
#Hindi_corpus = Hindi_corpus.read().splitlines()
#print('Hindi Corpus length is ',len(Hindi_corpus))

English Corpus length is  10000
Hindi Corpus length is  10000


In [3]:
#joint_Corpus_file = open('joint_Corpus.txt','w',encoding='utf-8')
#for i in range(10000):
#    joint_Corpus_file.write(Eng_corpus[i] + '|' +  Hindi_corpus[i] + '\n')
#joint_Corpus_file.close()
    

In [4]:
eng_hi_Corpus = open("joint_Corpus.txt","r", encoding='utf-8')
eng_hi_Corpus = eng_hi_Corpus.read().splitlines()
df = pd.DataFrame(eng_hi_Corpus)
X_train, X_test = train_test_split(df[0:10000], test_size=0.1, random_state=0)

In [5]:
eng_hi_train = np.array(X_train)
eng_hi_test = np.array(X_test)

### create train file for English and Hindi by splitting the text by "!"
eng_Corpus_train = open("eng_Corpus_train.txt","w", encoding='utf-8')
hin_Corpus_train = open("hin_Corpus_train.txt","w", encoding='utf-8')

for i in range(eng_hi_train.shape[0]):
    eng_Corpus_train.write(eng_hi_train[i][0].split('|')[0] + '\n')
    hin_Corpus_train.write(eng_hi_train[i][0].split('|')[1] + '\n')
  
eng_Corpus_train.close()
hin_Corpus_train.close() 

### create test file for English and Hindi by splitting the text by "!"

eng_Corpus_test = open("eng_Corpus_test.txt","w", encoding='utf-8')
hin_Corpus_test = open("hin_Corpus_test.txt","w", encoding='utf-8')

for i in range(eng_hi_test.shape[0]):
    eng_Corpus_test.write(eng_hi_test[i][0].split('|')[0] + '\n')
    hin_Corpus_test.write(eng_hi_test[i][0].split('|')[1] + '\n')
  
eng_Corpus_test.close()
hin_Corpus_test.close() 

In [6]:
from keras.models import Model
from keras.models import load_model,save_model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Activation, dot, concatenate, Bidirectional, Attention
from keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import nltk

In [7]:
BATCH_SIZE = 32
EPOCHS = 20
LSTM_NODES =128
NUM_SENTENCES = 9000
MAX_NUM_WORDS = 10000
EMBEDDING_SIZE = 100

In [8]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

En_Corpus = open('eng_Corpus_train.txt','r',encoding = 'utf-8')
cnt = 0 
for line in En_Corpus.readlines():
    #print(line)
    if cnt < NUM_SENTENCES:
      input_sentences.append(line.strip('\n'))
    cnt = cnt+1
En_Corpus.close()

Hi_Corpus = open('hin_Corpus_train.txt','r',encoding = 'utf-8')
cnt = 0 
for line in Hi_Corpus.readlines():
  if cnt < NUM_SENTENCES:
    output_sentences.append(line.strip('\n') + ' <EOS>')
    output_sentences_inputs.append('<SOS> '+line.strip('\n') )
  cnt = cnt +1
Hi_Corpus.close()

In [9]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))

num samples input: 9000
num samples output: 9000


In [10]:
input_tokenizer = Tokenizer(filters='')
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)
num_words_input = len(word2idx_inputs) + 1

Total unique words in the input: 8219
Length of longest sentence in input: 5


In [11]:
output_tokenizer = Tokenizer(filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 8270
Length of longest sentence in the output: 29


In [12]:
input_text = input_sentences
target_text_input = output_sentences_inputs
target_text_output = output_sentences

input_integer_seq = input_tokenizer.texts_to_sequences(input_text)
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
         
output_input_integer_seq = output_tokenizer.texts_to_sequences(target_text_input)
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
          
output_integer_seq = output_tokenizer.texts_to_sequences(target_text_output)
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
          
decoder_targets_one_hot = np.zeros((len(input_sentences),max_out_len,num_words_output),dtype='float32')

#print('decoder_targets_one_hot ',decoder_targets_one_hot.shape)


for i, d in enumerate(decoder_output_sequences):
  for t, word in enumerate(d):
    decoder_targets_one_hot[i, t, word] = 1
          

In [13]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
encoder_embedding = Embedding(num_words_input, LSTM_NODES)

x = encoder_embedding(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_sequences = True, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, decoder_states_h, decoder_states_c = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)
decoder_states = [decoder_states_h,decoder_states_c]

decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)

opt = RMSprop(learning_rate=0.001)
model.compile(
    #optimizer='rmsprop',
    optimizer = opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 29)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 5, 128)       1052160     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 29, 128)      1058688     input_2[0][0]                    
_______________________________________________________________________________________

In [15]:
callback = EarlyStopping(monitor='val_loss', patience=3)

r = model.fit([encoder_input_sequences, decoder_input_sequences],decoder_targets_one_hot,
              epochs=50,
              validation_split=0.2,
              callbacks=[callback])

Epoch 1/50
225/225 [==============================] - 17s 77ms/step - loss: 1.7379 - accuracy: 0.7999 - val_loss: 1.2811 - val_accuracy: 0.8227
Epoch 2/50
225/225 [==============================] - 16s 72ms/step - loss: 1.2343 - accuracy: 0.8225 - val_loss: 1.2435 - val_accuracy: 0.8250
Epoch 3/50
225/225 [==============================] - 16s 73ms/step - loss: 1.1819 - accuracy: 0.8269 - val_loss: 1.2100 - val_accuracy: 0.8310
Epoch 4/50
225/225 [==============================] - 16s 73ms/step - loss: 1.1283 - accuracy: 0.8340 - val_loss: 1.1733 - val_accuracy: 0.8372
Epoch 5/50
225/225 [==============================] - 16s 72ms/step - loss: 1.0846 - accuracy: 0.8400 - val_loss: 1.1489 - val_accuracy: 0.8426
Epoch 6/50
225/225 [==============================] - 16s 73ms/step - loss: 1.0488 - accuracy: 0.8447 - val_loss: 1.1339 - val_accuracy: 0.8453
Epoch 7/50
225/225 [==============================] - 16s 73ms/step - loss: 1.0176 - accuracy: 0.8482 - val_loss: 1.1224 - val_accuracy:

In [16]:
model.save('Seq2Seq_assignment2_v1.h5')